In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')
df.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division((x['total_revenue'] * 100), x['measurable_impressions']) * 1000 , axis=1)

In [ ]:
X_train = df.loc[df['date'] <= '2019-06-21 00:00:00'].drop(columns=['date', 'total_revenue', 'order_id' , 'line_item_type_id', 
                                                                    'integration_type_id' , 'revenue_share_percent'])
X_train = X_train[(X_train['CPM'] >= 0) & (X_train['CPM'] <= X_train['CPM'].quantile(.95))]
y_train = X_train['CPM']
X_train.drop(columns=['CPM'], inplace=True)

X_test = df.loc[df['date'] > '2019-06-21 00:00:00'].drop(columns=['date', 'total_revenue', 'order_id' , 'line_item_type_id', 
                                                                  'integration_type_id' , 'revenue_share_percent'])
X_test = X_test[(X_test['CPM'] >= 0) & (X_test['CPM'] <= X_test['CPM'].quantile(.95))]
y_test = X_test['CPM']
X_test.drop(columns=['CPM'], inplace=True)

In [ ]:
reg = xgb.XGBRegressor(random_state=0)
reg.fit(X_train, y_train, eval_metric='rmse')
pred = reg.predict(X_test)
mean_squared_error(y_test, pred)

In [ ]:
# xg_reg = xgb.XGBRegressor(random_state=0)
# params = {
#         'subsample': [0.8, 1.0],
#         'colsample_bytree': [0.8, 1.0],
#         'max_depth': [3, 5],
#         'n_estimators': [100, 300]
#         }

# grid_search = GridSearchCV(xg_reg, params, cv=5, n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')

# grid_search.fit(X_train, y_train)

In [ ]:
# grid_search.best_params_
# {'colsample_bytree': 1.0,
#  'max_depth': 5,
#  'n_estimators': 300,
#  'subsample': 1.0}

In [ ]:
# mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test))
# 4715.629061662551

In [ ]:
# xg_reg = xgb.XGBRegressor(random_state=0)
# params = {
#         'max_depth': [3, 5, 10],
#         'n_estimators': [100, 300, 500]
#         }

# grid_search = GridSearchCV(xg_reg, params, cv=5, n_jobs=-1, verbose=True, scoring='neg_mean_squared_error')

# grid_search.fit(X_train, y_train)

In [ ]:
# grid_search.best_params_
# {'max_depth': 10, 'n_estimators': 100}

In [ ]:
# mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test))
# 4415.274793428106

In [ ]:
reg = xgb.XGBRegressor(random_state=0, max_depth=10, n_estimators=100)
reg.fit(X_train, y_train, eval_metric='rmse')
pred = reg.predict(X_test)
mean_squared_error(y_test, pred)

# MSE = 4517